1. Подготовка данных \
проверить наличие пропусков и выбросов \
привести категориальные признаки к числовым \
вычислить парные корреляции признаков

2. Многомерная линейная регрессия \
Построить модель линейной регрессии и подобрать параметры: \
аналитически (реализовать самому) \
численно, с помощью методов градиентного спуска (реализовать самому)

3. Добавление регуляризации \
Модифицировать линейную модель путем добавления регуляризационного слагаемого. Найти оптимальные веса: \
аналитически \
численно

4. Оценка обобщающей способности \
Сравнить между собой модели на тестовых данных по среднему квадрату ошибки: \
константную - прогноз средним значением \
из пункта 2 \
из пункта 3